In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
🔽 모델 및 데이터셋 선택

In [ ]:
MODEL_NAME = 'patchcore'
DATASET_NAME = 'TR-bearing'

🔽 학습 파라미터 설정

In [ ]:
IMAGE_SIZE = (80,64)
SPLIT_RATIO = 0.2
MAX_EPOCHS = 300
SEED = 100
#BATCH_SIZE = 32



---



# 준비

In [ ]:
%%capture
%cd /content/drive/MyDrive/anomaly_detection/ANOMALIB
!pip install -e . -q

In [ ]:
import os, yaml, warnings
from pytorch_lightning import Trainer, seed_everything
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.utils.callbacks import get_callbacks
from anomalib.utils.loggers import configure_logger, get_experiment_logger
configure_logger('ERROR')
warnings.filterwarnings("ignore")

ModuleNotFoundError: ignored

> 경로 설정

In [ ]:
DATA_ROOT = '../datasets/canAIry/'+DATASET_NAME
RESULT_ROOT = '../results'
WEIGHTS=RESULT_ROOT+'/'+MODEL_NAME+'/'+DATASET_NAME+"/weights"
CONFIG_PATHS = 'anomalib/models'
CONFIG_YAML = CONFIG_PATHS + '/' + MODEL_NAME + '/' + DATASET_NAME + '.yaml'
MODEL_CONFIG_PAIRS = {
    'patchcore': f'{CONFIG_PATHS}/patchcore/config.yaml',
    'padim':     f'{CONFIG_PATHS}/padim/config.yaml',
    'cflow':     f'{CONFIG_PATHS}/cflow/config.yaml',
    'dfkde':     f'{CONFIG_PATHS}/dfkde/config.yaml',
    'dfm':       f'{CONFIG_PATHS}/dfm/config.yaml',
    'ganomaly':  f'{CONFIG_PATHS}/ganomaly/config.yaml',
    'stfpm':     f'{CONFIG_PATHS}/stfpm/config.yaml',
    'fastflow':  f'{CONFIG_PATHS}/fastflow/config.yaml',
    'draem':     f'{CONFIG_PATHS}/draem/config.yaml',
    'reverse_distillation': f'{CONFIG_PATHS}/reverse_distillation/config.yaml',
}

> 설정 파일(.yaml) 생성

In [ ]:
with open(MODEL_CONFIG_PAIRS[MODEL_NAME]) as f:
  default = yaml.safe_load(f)
  default['project']['path'] = RESULT_ROOT
  default['project']['seed'] = SEED
  default['dataset']['name'] = DATASET_NAME
  default["dataset"]["path"] = DATA_ROOT
  default["dataset"]["image_size"] = IMAGE_SIZE
  default['dataset']['split_ratio'] = SPLIT_RATIO
  default["dataset"]["normal_test_dir"] = 'normal/test'
  default['dataset']['normal_dir'] = 'normal'
  default['dataset']['abnormal_dir'] = 'abnormal'
  default["dataset"]["format"] = 'folder'
  default['dataset']['task'] = 'classification'
  default['dataset']['mask'] = None 
  default['dataset']['extensions'] = None
  default['dataset']['category'] = None
  default['dataset']['create_validation_set'] = True
  default['trainer']['detect_anomaly'] = True
  default['trainer']['auto_scale_batch_size'] = True
  default['trainer']['auto_lr_find'] = True
  default['trainer']['auto_select_gpus'] = True
  default['trainer']['move_metrics_to_cpu'] = True
  default['trainer']['max_epochs'] = MAX_EPOCHS
  #default['trainer']['reload_dataloaders_every_n_epochs'] = 3
  #default['dataset']['train_batch_size'] = BATCH_SIZE
  # default['logging']['logger'] = 'wandb'
  # default['logging']['log_graph'] = True
with open(CONFIG_YAML, 'w') as f:
  yaml.safe_dump(default, f, default_flow_style=False)

> 모델 및 데이터셋 로드

In [ ]:
config = get_configurable_parameters(model_name=MODEL_NAME, config_path=CONFIG_YAML)
seed_everything(config.project.seed)
model = get_model(config)
datamodule = get_datamodule(config)
trainer = Trainer(**config.trainer, logger=get_experiment_logger(config), callbacks=get_callbacks(config))

# 학습

## 처음

In [ ]:
trainer.fit(model=model, datamodule=datamodule)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AdaptiveThreshold        | 0     
1 | pixel_threshold       | AdaptiveThreshold        | 0     
2 | model                 | PatchcoreModel           | 24.9 M
3 | image_metrics         | AnomalibMetricCollection | 0     
4 | pixel_metrics         | AnomalibMetricCollection | 0     
5 | normalization_metrics | MinMax                   | 0     
-------------------------------------------------------------------
24.9 M    Trainable params
0         Non-trainable params
24.9 M    Total params
99.450    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

RuntimeError: ignored

## 이어서

In [ ]:
trainer.fit(model=model, datamodule=datamodule, ckpt_path=WEIGHTS+'/'+os.listdir(WEIGHTS)[0])